In [ ]:
import math
import json
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objs as go

with open("2987228299.json", "r") as fp:
    jdata = json.load(fp)

df = pd.DataFrame(jdata['data'][0]['values'],
                  columns=jdata['data'][0]['fields'])

df = df.join(pd.DataFrame(df.latlng.tolist(), columns=['geo_lat', 'geo_lon']))
df.drop('latlng', axis=1, inplace=True)

df['time'] = df['time'].astype('datetime64[s]')

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371000  # metre

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c

    return d

def get_stop_points():
    R = 0.4  # радиус
    len_df = df.shape[0]
    stop_point = []
    prev_dst = 0
    max_count_points = 4
    max_point = []
    for n, row in df.iterrows():
        n_min = min(n, 6)
        n_max = min(len_df - n, 6)
        cur_point = [row['geo_lat'], row['geo_lon']]
        count_points = 0
        lst_range = list(range(-n_min,0))+list(range(1,n_max))
        for i in lst_range:
            point = df.loc[n + i, ['geo_lat', 'geo_lon']].values
            dst = distance(point, cur_point)
            if dst < R:
                count_points += 1
        if count_points <= 4:
            max_points = 4
        elif count_points > max_points:
            max_points = count_points
            max_point = cur_point
        else:
            if max_point:
                stop_point.append(max_point)
            max_point = []
    return pd.DataFrame(stop_point, columns=['geo_lat', 'geo_lon'])

df_stop_points = get_stop_points()

fig = go.Figure(
    go.Scattermapbox(legendgroup="group",
                     name='Зменинная тропа',
                     lat=df['geo_lat'],
                     lon=df['geo_lon'],
                     mode='lines+markers',
                     marker=dict(
                         colorbar=dict(title="Высота над уровнем моря"),
                         color=df['elevation'],
                         colorscale='inferno')))
fig.add_scattermapbox(legendgroup="group",
                      name='Точки остановок',
                      lat=df_stop_points['geo_lat'],
                      lon=df_stop_points['geo_lon'],
                      mode='markers',
                      marker=dict(color='red', size=10))

map_center = go.layout.mapbox.Center(
    lat=(df['geo_lat'].max() + df['geo_lat'].min()) / 2,
    lon=(df['geo_lon'].max() + df['geo_lon'].min()) / 2)
fig.update_layout(title='Змеинная тропа',
                  legend_orientation="h",
                  mapbox_style="open-street-map",
                  mapbox=dict(center=map_center, zoom=15))

fig.show()

print(
    f"Максимальный перепад высот = {df['elevation'].max() - df['elevation'].min()} метра"
)

Максимальный перепад высот = 332 метра


**Вопрос 1.** Это трек подъёма или спуска? Аргументируйте ответ.
**Ответ**: Подъема. Видно на графике.

**Вопрос 2.** Каков максимальный перепад высот? Т.е. разница между минимальной и максимальной высотой.
**Ответ** выше.